In [11]:
import datetime
import backtrader as bt
from strategies import *

cerebro = bt.Cerebro()

# Set data parameters and add to Cerebro

data = bt.feeds.YahooFinanceCSVData(dataname='TSLA.csv',
                                    fromdate=datetime.datetime(2016, 1, 1),
                                    todate=datetime.datetime(2017, 12, 25))

#settings for out-of-sample data
#fromdate=datetime.datetime(2018, 1, 1),
#todate=datetime.datetime(2019, 12, 25))

cerebro.adddata(data)

# Default position size

cerebro.addsizer(bt.sizers.SizerFix, stake=3)

if __name__ == '__main__':
    #Run Cerebro Engine
    start_portfolio_value = cerebro.broker.getvalue()
    cerebro.run() 
    end_portfolio_value = cerebro.broker.getvalue()
    pnl = end_portfolio_value - start_portfolio_value 
    print('Starting Portfolio Value: %.2f' % start_portfolio_value) 
    print('Final Portfolio Value: %.2f' % end_portfolio_value) 
    print('PnL: %.2f' % pnl)
    
class MAcrossover(bt.Strategy): 
    
    #Moving average parameters
    params = (('pfast',20),('pslow',50),)
    
    def log(self, txt, dt=None):     
        dt = dt or self.datas[0].datetime.date(0)     
        print('%s, %s' % (dt.isoformat(), txt)) 
    
    
    def __init__(self):     
        self.dataclose = self.datas[0].close     
        # Order variable will contain ongoing order details/status
        self.order = None     
        # Instantiate moving averages     
        self.slow_sma = bt.indicators.MovingAverageSimple(self.datas[0],                                       
                        period=self.params.pslow)     
        self.fast_sma = bt.indicators.MovingAverageSimple(self.datas[0], 
                        period=self.params.pfast)
        
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            #Active Buy/Sell order submitted/accepted - Nothing to do
            return
        #Check if an order has been completed
        #Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():             
                self.log('BUY EXECUTED, %.2f' % order.executed.price)         
            elif order.issell():             
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')     
        #Reset orders  
        self.order = None
    
    
    def next(self):
        if self.order:
            return
        #Check if we are in the market
        if not self.position:
        #We are not in the market, look for a signal to OPEN trades
            #If the 20 SMA is above the 50 SMA
            if self.fast_sma[0] > self.slow_sma[0] and self.fast_sma[-1] < self.slow_sma[-1]:
                 self.log('BUY CREATE, %.2f' % self.dataclose[0])
                 #Keep track of the created order to avoid a 2nd order
                 self.order = self.buy()
            #Otherwise if the 20 SMA is below the 50 SMA
            elif self.fast_sma[0] < self.slow_sma[0] and self.fast_sma[-1] > self.slow_sma[-1]:
                self.log('SELL CREATE, %.2f' % self.dataclose[0])
            #Keep track of the created order to avoid a 2nd order
            self.order = self.sell()
        else:
            # We are already in the market, look for a signal to CLOSE trades
            if len(self) >= (self.bar_executed + 5):
                self.log('CLOSE CREATE, %.2f' % self.dataclose[0])
                self.order = self.close()

# Add strategy to Cerebro

cerebro.addstrategy(MAcrossover)

Starting Portfolio Value: 10000.00
Final Portfolio Value: 10000.00
PnL: 0.00


1